In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [106]:
ls

bouncing_ball.png  experiment1/  gdrive/  sample_data/


In [2]:
cd gdrive/

/content/gdrive


In [3]:
ls

MyDrive/


In [4]:
cd MyDrive/

/content/gdrive/MyDrive


In [5]:
pip install torchdiffeq

In [113]:
pip install tensorflow_probability

In [115]:
pip install hickle==3.4

  Created wheel for hickle: filename=hickle-3.4.0-py3-none-any.whl size=31625 sha256=d21d9a1aa0baf860f9dc3ced05c167c969ba2528b5c0bca0499130b16733172b
  Stored in directory: /root/.cache/pip/wheels/8b/8e/90/9662ef4bc699029970c0b7957b9bbcf1b58ffcd063ebba9b9d
Successfully built hickle


In [116]:
pip install python==3.7

ERROR: Could not find a version that satisfies the requirement python==3.7 (from versions: none)
ERROR: No matching distribution found for python==3.7


In [117]:
!git clone https://github.com/cagatayyildiz/ODE2VAE.git

Cloning into 'ODE2VAE'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 218 (delta 2), reused 1 (delta 0), pack-reused 212
Receiving objects: 100% (218/218), 78.33 MiB | 23.65 MiB/s, done.
Resolving deltas: 100% (98/98), done.


In [118]:
cd ODE2VAE/

/content/gdrive/My Drive/ODE2VAE


In [119]:
ls

images/  ode2vae_poster.pdf  scripts/  torch_bnn.py              train.py
model/   README.md           test.py   torch_ode2vae_minimal.py


In [120]:
ls

images/             README.md         test.py                   train.py
model/              rot-mnist-3s.mat  torch_bnn.py
ode2vae_poster.pdf  scripts/          torch_ode2vae_minimal.py


In [123]:
cd scripts/

/content/gdrive/My Drive/ODE2VAE/scripts


In [124]:
ls

test_bballs.sh            test_mocap_single.sh       train_mocap_many.sh
test_mnist_nonuniform.sh  train_bballs.sh            train_mocap_single.sh
test_mnist.sh             train_mnist_nonuniform.sh
test_mocap_many.sh        train_mnist.sh


In [133]:
!train_bballs.sh

/bin/bash: train_bballs.sh: command not found


In [6]:
import os
import argparse
import time
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

parser = argparse.ArgumentParser('ODE demo')
parser.add_argument('--method', type=str, choices=['dopri5', 'adams'], default='dopri5')
parser.add_argument('--data_size', type=int, default=1000)
parser.add_argument('--batch_time', type=int, default=10)
parser.add_argument('--batch_size', type=int, default=20)
parser.add_argument('--niters', type=int, default=2000)
parser.add_argument('--test_freq', type=int, default=20)
parser.add_argument('--viz', action='store_true')
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--adjoint', action='store_true',default=True)
args = parser.parse_args(args=[])

In [7]:
args.adjoint

True

In [8]:
if args.adjoint:
    from torchdiffeq import odeint_adjoint as odeint
else:
    from torchdiffeq import odeint

device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

In [9]:
true_y0 = torch.tensor([[2., 0.]]).to(device)
t = torch.linspace(0., 25., args.data_size).to(device)
true_A = torch.tensor([[-0.1, 2.0], [-2.0, -0.1]]).to(device)

In [10]:
t

tensor([ 0.0000,  0.0250,  0.0501,  0.0751,  0.1001,  0.1251,  0.1502,  0.1752,
         0.2002,  0.2252,  0.2503,  0.2753,  0.3003,  0.3253,  0.3504,  0.3754,
         0.4004,  0.4254,  0.4505,  0.4755,  0.5005,  0.5255,  0.5506,  0.5756,
         0.6006,  0.6256,  0.6507,  0.6757,  0.7007,  0.7257,  0.7508,  0.7758,
         0.8008,  0.8258,  0.8509,  0.8759,  0.9009,  0.9259,  0.9510,  0.9760,
         1.0010,  1.0260,  1.0511,  1.0761,  1.1011,  1.1261,  1.1512,  1.1762,
         1.2012,  1.2262,  1.2513,  1.2763,  1.3013,  1.3263,  1.3514,  1.3764,
         1.4014,  1.4264,  1.4515,  1.4765,  1.5015,  1.5265,  1.5516,  1.5766,
         1.6016,  1.6266,  1.6517,  1.6767,  1.7017,  1.7267,  1.7518,  1.7768,
         1.8018,  1.8268,  1.8519,  1.8769,  1.9019,  1.9269,  1.9520,  1.9770,
         2.0020,  2.0270,  2.0521,  2.0771,  2.1021,  2.1271,  2.1522,  2.1772,
         2.2022,  2.2272,  2.2523,  2.2773,  2.3023,  2.3273,  2.3524,  2.3774,
         2.4024,  2.4274,  2.4525,  2.47

In [11]:
class Lambda(nn.Module):

    def forward(self, t, y):
        return torch.mm(y**3, true_A)

with torch.no_grad():
    true_y = odeint(Lambda(), true_y0, t, method='dopri5')

In [12]:
def get_batch():
    s = torch.from_numpy(np.random.choice(np.arange(args.data_size - args.batch_time, dtype=np.int64), args.batch_size, replace=False))
    batch_y0 = true_y[s]  # (M, D)
    batch_t = t[:args.batch_time]  # (T)
    batch_y = torch.stack([true_y[s + i] for i in range(args.batch_time)], dim=0)  # (T, M, D)
    return batch_y0.to(device), batch_t.to(device), batch_y.to(device)


In [14]:
s = torch.from_numpy(np.random.choice(np.arange(args.data_size - args.batch_time, dtype=np.int64), args.batch_size, replace=False))
s.shape

torch.Size([20])

In [39]:
batch_y = torch.stack([true_y[s + i] for i in range(args.batch_time)], dim=0)  # (T, M, D)

In [43]:
batch_y.shape

torch.Size([10, 20, 1, 2])

In [193]:
def makedirs(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)

In [194]:
if args.viz:
    makedirs('png')
    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=(12, 4), facecolor='white')
    ax_traj = fig.add_subplot(131, frameon=False)
    ax_phase = fig.add_subplot(132, frameon=False)
    ax_vecfield = fig.add_subplot(133, frameon=False)
    plt.show(block=False)


In [195]:
def visualize(true_y, pred_y, odefunc, itr):

    if args.viz:

        ax_traj.cla()
        ax_traj.set_title('Trajectories')
        ax_traj.set_xlabel('t')
        ax_traj.set_ylabel('x,y')
        ax_traj.plot(t.cpu().numpy(), true_y.cpu().numpy()[:, 0, 0], t.cpu().numpy(), true_y.cpu().numpy()[:, 0, 1], 'g-')
        ax_traj.plot(t.cpu().numpy(), pred_y.cpu().numpy()[:, 0, 0], '--', t.cpu().numpy(), pred_y.cpu().numpy()[:, 0, 1], 'b--')
        ax_traj.set_xlim(t.cpu().min(), t.cpu().max())
        ax_traj.set_ylim(-2, 2)
        ax_traj.legend()

        ax_phase.cla()
        ax_phase.set_title('Phase Portrait')
        ax_phase.set_xlabel('x')
        ax_phase.set_ylabel('y')
        ax_phase.plot(true_y.cpu().numpy()[:, 0, 0], true_y.cpu().numpy()[:, 0, 1], 'g-')
        ax_phase.plot(pred_y.cpu().numpy()[:, 0, 0], pred_y.cpu().numpy()[:, 0, 1], 'b--')
        ax_phase.set_xlim(-2, 2)
        ax_phase.set_ylim(-2, 2)

        ax_vecfield.cla()
        ax_vecfield.set_title('Learned Vector Field')
        ax_vecfield.set_xlabel('x')
        ax_vecfield.set_ylabel('y')

        y, x = np.mgrid[-2:2:21j, -2:2:21j]
        dydt = odefunc(0, torch.Tensor(np.stack([x, y], -1).reshape(21 * 21, 2)).to(device)).cpu().detach().numpy()
        mag = np.sqrt(dydt[:, 0]**2 + dydt[:, 1]**2).reshape(-1, 1)
        dydt = (dydt / mag)
        dydt = dydt.reshape(21, 21, 2)

        ax_vecfield.streamplot(x, y, dydt[:, :, 0], dydt[:, :, 1], color="black")
        ax_vecfield.set_xlim(-2, 2)
        ax_vecfield.set_ylim(-2, 2)

        fig.tight_layout()
        plt.savefig('png/{:03d}'.format(itr))
        plt.draw()
        plt.pause(0.001)


In [196]:
class ODEFunc(nn.Module):

    def __init__(self):
        super(ODEFunc, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(2, 50),
            nn.Tanh(),
            nn.Linear(50, 2),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.1)
                nn.init.constant_(m.bias, val=0)

    def forward(self, t, y):
        return self.net(y**3)

In [197]:
class RunningAverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, momentum=0.99):
        self.momentum = momentum
        self.reset()

    def reset(self):
        self.val = None
        self.avg = 0

    def update(self, val):
        if self.val is None:
            self.avg = val
        else:
            self.avg = self.avg * self.momentum + val * (1 - self.momentum)
        self.val = val

In [198]:
if __name__ == '__main__':
  ii = 0
  func = ODEFunc().to(device)
  optimizer = optim.RMSprop(func.parameters(), lr=1e-3)
  end = time.time()
  time_meter = RunningAverageMeter(0.97)
    
  loss_meter = RunningAverageMeter(0.97)

  for itr in range(1, args.niters + 1):
        optimizer.zero_grad()
        batch_y0, batch_t, batch_y = get_batch()
        pred_y = odeint(func, batch_y0, batch_t).to(device)
        loss = torch.mean(torch.abs(pred_y - batch_y))
        loss.backward()
        optimizer.step()

        time_meter.update(time.time() - end)
        loss_meter.update(loss.item())

        if itr % args.test_freq == 0:
            with torch.no_grad():
                pred_y = odeint(func, true_y0, t)
                loss = torch.mean(torch.abs(pred_y - true_y))
                print('Iter {:04d} | Total Loss {:.6f}'.format(itr, loss.item()))
                visualize(true_y, pred_y, func, ii)
                ii += 1

        end = time.time()






Iter 0020 | Total Loss 0.628735
Iter 0040 | Total Loss 0.710426
Iter 0060 | Total Loss 0.957831
Iter 0080 | Total Loss 0.793954
Iter 0100 | Total Loss 0.242254
Iter 0120 | Total Loss 0.770080
Iter 0140 | Total Loss 0.368988
Iter 0160 | Total Loss 0.741123
Iter 0180 | Total Loss 0.421135
Iter 0200 | Total Loss 0.297264
Iter 0220 | Total Loss 0.360812
Iter 0240 | Total Loss 0.416897
Iter 0260 | Total Loss 0.194766
Iter 0280 | Total Loss 0.238816
Iter 0300 | Total Loss 0.191598
Iter 0320 | Total Loss 0.670691
Iter 0340 | Total Loss 0.376499
Iter 0360 | Total Loss 0.188519
Iter 0380 | Total Loss 0.224572
Iter 0400 | Total Loss 0.475061
Iter 0420 | Total Loss 0.702798
Iter 0440 | Total Loss 0.166920
Iter 0460 | Total Loss 0.673048
Iter 0480 | Total Loss 0.231554
Iter 0500 | Total Loss 0.195185
Iter 0520 | Total Loss 0.305168
Iter 0540 | Total Loss 0.319387
Iter 0560 | Total Loss 0.231433
Iter 0580 | Total Loss 0.224575
Iter 0600 | Total Loss 0.164221
Iter 0620 | Total Loss 0.604486
Iter 064

In [175]:
s = torch.from_numpy(np.random.choice(np.arange(args.data_size - args.batch_time, dtype=np.int64), args.batch_size, replace=False))

In [177]:
batch_y0.shape

torch.Size([20, 1, 2])

In [178]:
batch_t.shape

torch.Size([10])

In [179]:
batch_y.shape

torch.Size([10, 20, 1, 2])

In [181]:
ls

test_bballs.sh            test_mocap_single.sh       train_mocap_many.sh
test_mnist_nonuniform.sh  train_bballs.sh            train_mocap_single.sh
test_mnist.sh             train_mnist_nonuniform.sh
test_mocap_many.sh        train_mnist.sh


In [1]:
ls

sample_data/


In [ ]:
1+1